In [1]:
import os
import requests
import pandas as pd
import urllib.request
from tqdm import tqdm
from io import StringIO
from bs4 import BeautifulSoup

In [59]:
master_foldername = '/home/jingxin/data/master/'
exomol_all_path = master_foldername + 'exomol__20200720.all'

In [76]:
def get_new_def():
    list_files = []

    urlpage_mol = 'https://www.exomol.com/data/molecules/'
    r_mol = requests.get(urlpage_mol)
    if r_mol.status_code==200:
        page_mol = urllib.request.urlopen(urlpage_mol)
    soup_mol = BeautifulSoup(page_mol, 'html.parser')
    list_group_mol = soup_mol.find_all('div', attrs={'class': 'list-group'})
    list_molecule = []
    for i_mol in range(len(list(list_group_mol))):
        n_mol = len(list(list(list_group_mol)[i_mol]))
        list_group_item_mol = list(list(list_group_mol)[i_mol])[1:n_mol:2]
        str_group_mol = str(list_group_item_mol).split('"')
        list_molecule += str_group_mol[3:len(str_group_mol):4]

    for index_mol in tqdm(range(len(list_molecule))):    
        urlpage_iso = urlpage_mol + list_molecule[index_mol]
        print(list_molecule[index_mol])
        r_iso = requests.get(urlpage_iso)
        if r_iso.status_code==200:
            page_iso = urllib.request.urlopen(urlpage_iso)
        soup_iso = BeautifulSoup(page_iso, 'html.parser')
        list_group_iso = soup_iso.find_all('div', attrs={'class': 'list-group'})
        list_iso = []
        for i_iso in range(len(list(list_group_iso))):
            n_iso = len(list(list(list_group_iso)[i_iso]))
            list_group_item_iso = list(list(list_group_iso)[i_iso])[1:n_iso:2]
            str_group_iso = str(list_group_item_iso).split('"')
            list_iso += str_group_iso[3:len(str_group_iso):4]

        if (len(list_iso) != 0):
            for index_iso in range(len(list_iso)):
                urlpage_dataset = urlpage_iso + '/' + list_iso[index_iso]
                r_dataset = requests.get(urlpage_dataset)
                if r_dataset.status_code==200:
                    page_dataset = urllib.request.urlopen(urlpage_dataset)
                soup_dataset = BeautifulSoup(page_dataset, 'html.parser')
                list_group_dataset = soup_dataset.find_all('div', attrs={'class': 'list-group'})
                list_datasets = []
                for i_dataset in range(len(list(list_group_dataset))):
                    n_dataset = len(list(list(list_group_dataset)[i_dataset]))
                    list_group_item_dataset = list(list(list_group_dataset)[i_dataset])[1:n_dataset:2]
                    str_group_dataset = str(list_group_item_dataset).split('"')
                    list_datasets += str_group_dataset[3:len(str_group_dataset):4]
                list_dataset = list_datasets
                a = 'list-group-item link-list-group-item recommended'
                b = 'list-group-item link-list-group-item'
                c = 'pull-right'
                if a in list_dataset:
                    list_dataset.remove('list-group-item link-list-group-item recommended')
                if b in list_dataset:
                    list_dataset.remove('list-group-item link-list-group-item')
                if c in list_dataset:
                    list_dataset.remove('pull-right')

                if (len(list_dataset) != 0):
                    for index_dataset in range(len(list_dataset)):
                        urlpage_file = urlpage_dataset + '/' + list_dataset[index_dataset]
                        r_file = requests.get(urlpage_file)
                        if r_file.status_code==200:
                            page_file = urllib.request.urlopen(urlpage_file)
                        soup_file = BeautifulSoup(page_file, 'html.parser')
                        list_group_file = soup_file.find_all('div', attrs={'class': 'list-group'})

                        if (len(list_group_file) != 0):
                            for i_file in range(len(list(list_group_file))):
                                n_file = len(list(list(list_group_file)[i_file]))
                                list_group_item_file = list(list(list_group_file)[i_file])[1:n_file:2]
                                str_group_file = str(list_group_item_file).replace('/models','https://www.exomol.com/db').split('"')
                                list_files += str_group_file[3:len(str_group_file):4]

    file_list = [start_link+start_link.split('/')[5]+'__'+start_link.split('/')[6]+'.def' for start_link in list_files]
    new_def_url = pd.DataFrame()
    new_def_url['new'] = file_list
    return new_def_url

In [36]:
def get_old_def():
    exomol_col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6']
    
    #################################################
    # Use online master file (last or new version). #
    # exomol_all_url = 'http://www.exomol.com/db/exomol.all'
    # content = requests.get(exomol_all_url).text.replace('#','')
    # exomol_all = pd.read_csv(StringIO(content), sep='\\s+', names=exomol_col_name, header=None)
    # Col1 = exomol_all['c1']
    # Col2 = exomol_all['c3']
    
    ##################################################
    # Use local master file (version when you save). #
    exomol_all = pd.read_csv(exomol_all_path, sep='\\s+', names=exomol_col_name, 
                             usecols=[0, 1, 2, 3, 4, 5, 6], header=None)
    Col1 = exomol_all['c2']
    Col2 = exomol_all['c4']
    
    ##################################################
    row = len(Col1)
    iso_slug = pd.DataFrame()
    iso_formula = pd.DataFrame()
    isotopologue = pd.DataFrame()
    molecule_single = pd.DataFrame()
    num_isotopologues = pd.DataFrame()

    for i in range(row):

        _iso_slug = exomol_all[Col1.isin(['Iso-slug'])]['c0'].values
        _iso_formula = exomol_all[Col1.isin(['IsoFormula'])]['c0'].values
        _isotopologue = exomol_all[Col1.isin(['Isotopologue'])]['c0'].values
        _molecule_single = exomol_all[Col1.isin(['Molecule'])]['c0'].values
        _num_isotopologues = exomol_all[Col2.isin(['isotopologues'])]['c0'].values

    iso_slug = pd.concat([iso_slug, pd.DataFrame(_iso_slug)])
    iso_formula = pd.concat([iso_formula, pd.DataFrame(_iso_formula)])
    isotopologue = pd.concat([isotopologue, pd.DataFrame(_isotopologue)])
    molecule_single = pd.concat([molecule_single, pd.DataFrame(_molecule_single)])
    num_isotopologues = pd.concat([num_isotopologues, pd.DataFrame(_num_isotopologues)])
    
    molecule_repeated = pd.DataFrame()
    molecule_num = len(molecule_single)

    for j in range(molecule_num):    
        molecule_repeated = pd.concat([molecule_repeated, pd.DataFrame((molecule_single.values[j] + ' ')
                                                                       *int(num_isotopologues.values[j]))])

    molecule_str = (str(molecule_repeated.values).replace("[['"," ")
                    .replace("']\n ['"," ").replace("']]"," ").replace("+","_p"))
    molecule = pd.read_csv(StringIO(molecule_str), sep='\s+', header=None)
    

    old_def_url_list = []
    old_def_num = len(iso_slug)
    for i in range(old_def_num):
        old_def_url_list.append('https://www.exomol.com/db/' + molecule[i] + '/'
                                + iso_slug.values[i] + '/'+ isotopologue.values[i] + '/'
                                + iso_slug.values[i] + '__' + isotopologue.values[i] + '.def')
    old_def_url = pd.DataFrame()
    old_def_url['old'] = pd.DataFrame(old_def_url_list)
    return old_def_url

In [37]:
def get_add_def(new_def_url, old_def_url):
    add_def_url = new_def_url[~(new_def_url['new'].isin(old_def_url['old']))].reset_index(drop=True)
    add_def_url.columns = ['add']
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', 1000)
    pd.set_option('display.max_colwidth', 1000)
    return add_def_url

In [60]:
def download_master_file():
    master_filename = 'exomol.all'
    if os.path.exists(master_foldername):
        pass
    else:
        os.makedirs(master_foldername, exist_ok=True)
    filename = os.path.join(master_foldername, master_filename)

    exomol_all_url = 'http://www.exomol.com/db/exomol.all'
    r = requests.get(exomol_all_url) 
    with open(filename, 'wb') as code:
        code.write(r.content)    


In [39]:
def get_invalid_urlpage(add_def_url):
    # Invalid URL page
    urlpages_list = []
    for i in tqdm(range(len(list(add_def_url['add'])))):
        urlpage = add_def_url['add'][i]
        r_add = requests.get(urlpage)
        if r_add.status_code!=200:
            urlpages_list.append(urlpage)
    urlpage = pd.DataFrame(urlpages_list)
    print('There are ', len(urlpage),' invalid url pages of def files.')
    return urlpage

In [77]:
new_def_url = get_new_def()
old_def_url = get_old_def()
add_def_url = get_add_def(new_def_url, old_def_url)

  0%|          | 0/92 [00:00<?, ?it/s]

MgH


  1%|          | 1/92 [00:06<10:29,  6.91s/it]

NaH


  2%|▏         | 2/92 [00:07<04:52,  3.25s/it]

...

 98%|█████████▊| 90/92 [01:55<00:02,  1.02s/it]

K


 99%|█████████▉| 91/92 [01:55<00:00,  1.21it/s]

Na


100%|██████████| 92/92 [01:55<00:00,  1.26s/it]


In [78]:
print('New version of master file ExoMol.all has ', int(len(new_def_url)), ' def files.')
new_def_url

New version of master file ExoMol.all has  250  def files.


,new
0,https://www.exomol.com/db/MgH/24Mg-1H/Yadin/24Mg-1H__Yadin.def
1,https://www.exomol.com/db/MgH/24Mg-1H/XAB/24Mg-1H__XAB.def
2,https://www.exomol.com/db/MgH/24Mg-1H/MoLLIST-MgH/24Mg-1H__MoLLIST-MgH.def
3,https://www.exomol.com/db/MgH/25Mg-1H/XAB/25Mg-1H__XAB.def
4,https://www.exomol.com/db/MgH/25Mg-1H/Yadin/25Mg-1H__Yadin.def
...,...
245,https://www.exomol.com/db/SiN/28Si-15N/SiNfull/28Si-15N__SiNfull.def
246,https://www.exomol.com/db/SiN/29Si-14N/SiNfull/29Si-14N__SiNfull.def
247,https://www.exomol.com/db/SiN/30Si-14N/SiNfull/30Si-14N__SiNfull.def
248,https://www.exomol.com/db/K/39K/Kurucz-K/39K__Kurucz-K.def


In [79]:
print('Old version of master file ExoMol.all has ', len(old_def_url), ' def files.')
old_def_url

Old version of master file ExoMol.all has  188  def files.


,old
0,https://www.exomol.com/db/H2O/2H2-16O/Hewitt/2H2-16O__Hewitt.def
1,https://www.exomol.com/db/H2O/1H2-16O/POKAZATEL/1H2-16O__POKAZATEL.def
2,https://www.exomol.com/db/H2O/1H2-18O/HotWat78/1H2-18O__HotWat78.def
3,https://www.exomol.com/db/H2O/1H2-17O/HotWat78/1H2-17O__HotWat78.def
4,https://www.exomol.com/db/H2O/1H-2H-16O/VTT/1H-2H-16O__VTT.def
...,...
183,https://www.exomol.com/db/SiO2/28Si-16O2/OYT3/28Si-16O2__OYT3.def
184,https://www.exomol.com/db/cis-P2H2/cis-31P2-1H2/OY-Cis/cis-31P2-1H2__OY-Cis.def
185,https://www.exomol.com/db/trans-P2H2/trans-31P2-1H-2H/OY-Trans/trans-31P2-1H-2H__OY-Trans.def
186,https://www.exomol.com/db/trans-P2H2/trans-31P2-1H2/OY-Trans/trans-31P2-1H2__OY-Trans.def


In [80]:
print('Master file ExoMol.all has ', len(add_def_url), ' new def files.')
add_def_url

Master file ExoMol.all has  72  new def files.


,add
0,https://www.exomol.com/db/MgH/24Mg-1H/XAB/24Mg-1H__XAB.def
1,https://www.exomol.com/db/MgH/24Mg-1H/MoLLIST-MgH/24Mg-1H__MoLLIST-MgH.def
2,https://www.exomol.com/db/MgH/25Mg-1H/XAB/25Mg-1H__XAB.def
3,https://www.exomol.com/db/MgH/26Mg-1H/XAB/26Mg-1H__XAB.def
4,https://www.exomol.com/db/CaH/40Ca-1H/MoLLIST-CaH/40Ca-1H__MoLLIST-CaH.def
...,...
67,https://www.exomol.com/db/SiN/28Si-15N/SiNfull/28Si-15N__SiNfull.def
68,https://www.exomol.com/db/SiN/29Si-14N/SiNfull/29Si-14N__SiNfull.def
69,https://www.exomol.com/db/SiN/30Si-14N/SiNfull/30Si-14N__SiNfull.def
70,https://www.exomol.com/db/K/39K/Kurucz-K/39K__Kurucz-K.def


In [61]:
download_master_file()

In [81]:
urlpage = get_invalid_urlpage(add_def_url)
urlpage

100%|██████████| 72/72 [00:01<00:00, 45.72it/s]

There are  9  invalid url pages of def files.


,0
0,https://www.exomol.com/db/MgH/24Mg-1H/MoLLIST-MgH/24Mg-1H__MoLLIST-MgH.def
1,https://www.exomol.com/db/CaH/40Ca-1H/MoLLIST-CaH/40Ca-1H__MoLLIST-CaH.def
2,https://www.exomol.com/db/FeH/56Fe-1H/Bernath/56Fe-1H__Bernath.def
3,https://www.exomol.com/db/HCl/1H-35Cl/PTY/1H-35Cl__PTY.def
4,https://www.exomol.com/db/HF/1H-19F/PTY/1H-19F__PTY.def
5,https://www.exomol.com/db/H2O/1H2-16O/VT/1H2-16O__VT.def
6,https://www.exomol.com/db/H2_p/1H2_p/ADJSAAM/1H2_p__ADJSAAM.def
7,https://www.exomol.com/db/H3_p/1H3_p/NT/1H3_p__NT.def
8,https://www.exomol.com/db/K/39K/Kurucz-K/39K__Kurucz-K.def
